# Machine Learning now

In [2]:
import pandas as pd
import os
import numpy as np
import sklearn 
import scipy.linalg as scplinag
from sklearn.neighbors import KDTree
from scipy.spatial import distance

# Whole dataset and 50 NN 

Attributes as in Weinmann, 2013, plus relative height (Weinmann, 2014) 

In [3]:
# Define a data frame with all my data# Define  
FILE_PATH_ScenarioB = r"../DATA/ML_datasets/ScenarioB"
FILE_PATH = r"../DATA"
FILE_NAME = r"/5_Data_ML_attributes_100NN.txt"
IMAGE_FILE_PATH = r"images"
df = pd.read_csv(FILE_PATH+FILE_NAME, delimiter=',')
df.head()

,X,Y,Z,class,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,radius_neighbourhood,relative_height,verticality
0,1905.494751,21122.037109,38.884586,2,0.102637,0.896414,0.000949,0.000250,0.999051,0.030035,0.005013,0.000500,2939.715359,0.201671,-2.497189,0.000847
1,1905.502930,21122.035156,38.883110,2,0.090709,0.908312,0.000979,0.000255,0.999021,0.030308,0.005067,0.000512,3304.676763,0.193955,-2.498982,0.000847
2,1905.512939,21122.033203,38.883949,2,0.124330,0.874710,0.000960,0.000248,0.999040,0.029584,0.004924,0.000512,3681.928229,0.187091,-2.498448,0.000847
3,1905.520874,21122.031250,38.882137,2,0.180508,0.818550,0.000941,0.000248,0.999059,0.029577,0.004925,0.000517,4059.958026,0.181094,-2.500576,0.000847
4,1905.531128,21122.029297,38.882839,2,0.220828,0.778263,0.000909,0.000249,0.999091,0.029796,0.004971,0.000511,4672.880330,0.172802,-2.500191,0.000902


In [4]:
columns_to_drop_10 = ["X", "Y", "Z", "radius_neighbourhood", "relative_height"]

for elem in columns_to_drop_10:
    df.drop([elem], axis=1, inplace=True)

df.head()

,class,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,verticality
0,2,0.102637,0.896414,0.000949,0.000250,0.999051,0.030035,0.005013,0.000500,2939.715359,0.000847
1,2,0.090709,0.908312,0.000979,0.000255,0.999021,0.030308,0.005067,0.000512,3304.676763,0.000847
2,2,0.124330,0.874710,0.000960,0.000248,0.999040,0.029584,0.004924,0.000512,3681.928229,0.000847
3,2,0.180508,0.818550,0.000941,0.000248,0.999059,0.029577,0.004925,0.000517,4059.958026,0.000847
4,2,0.220828,0.778263,0.000909,0.000249,0.999091,0.029796,0.004971,0.000511,4672.880330,0.000902


### I have made a mistake with the class names, therefore I need to re-name them now

In the file 3_CreateFinalClasses, I did not change the names of the classes properly. Therefore, I will have to re-name them now. 

Manually, not to confuse it.

In [5]:
# Road (second most points)
df["class"].where(df["class"] != 2, 1, inplace=True)
# Sidewalk
df["class"].where(df["class"] != 3, 2, inplace=True)
# Curb
df["class"].where(df["class"] != 4, 3, inplace=True)
# Building (most points)
df["class"].where(df["class"] != 5, 4, inplace=True)
# Other pole like objects
df["class"].where(df["class"] != 6, 5, inplace=True)
# Small poles
df["class"].where(df["class"] != 7, 6, inplace=True)
# Pedestrians
df["class"].where(df["class"] != 11, 7, inplace=True)
# 2 wheelers
df["class"].where(df["class"] != 12, 8, inplace=True)
# 4 wheelers
df["class"].where(df["class"] != 13, 9, inplace=True)
# Trees
df["class"].where(df["class"] != 14, 10, inplace=True)
# Potted plants
df["class"].where(df["class"] != 15, 11, inplace=True)

In [7]:
df.drop(["class"], axis=1, inplace=True)
df.head()

,lambda1,lambda2,lambda3,lambda4,lambda5,lambda6,lambda7,lambda8,local_density,verticality
0,0.102637,0.896414,0.000949,0.000250,0.999051,0.030035,0.005013,0.000500,2939.715359,0.000847
1,0.090709,0.908312,0.000979,0.000255,0.999021,0.030308,0.005067,0.000512,3304.676763,0.000847
2,0.124330,0.874710,0.000960,0.000248,0.999040,0.029584,0.004924,0.000512,3681.928229,0.000847
3,0.180508,0.818550,0.000941,0.000248,0.999059,0.029577,0.004925,0.000517,4059.958026,0.000847
4,0.220828,0.778263,0.000909,0.000249,0.999091,0.029796,0.004971,0.000511,4672.880330,0.000902


# Rename the columns to make it easier

In [8]:
df = df.rename(index=str, columns={"lambda1": "lambda1_100NN"})
df = df.rename(index=str, columns={"lambda2": "lambda2_100NN"})
df = df.rename(index=str, columns={"lambda3": "lambda3_100NN"})
df = df.rename(index=str, columns={"lambda4": "lambda4_100NN"})
df = df.rename(index=str, columns={"lambda5": "lambda5_100NN"})
df = df.rename(index=str, columns={"lambda6": "lambda6_100NN"})
df = df.rename(index=str, columns={"lambda7": "lambda7_100NN"})
df = df.rename(index=str, columns={"lambda8": "lambda8_100NN"})
df = df.rename(index=str, columns={"local_density": "local_density_100NN"})
df = df.rename(index=str, columns={"verticality": "verticality_100NN"})

df.head()

,lambda1_100NN,lambda2_100NN,lambda3_100NN,lambda4_100NN,lambda5_100NN,lambda6_100NN,lambda7_100NN,lambda8_100NN,local_density_100NN,verticality_100NN
0,0.102637,0.896414,0.000949,0.000250,0.999051,0.030035,0.005013,0.000500,2939.715359,0.000847
1,0.090709,0.908312,0.000979,0.000255,0.999021,0.030308,0.005067,0.000512,3304.676763,0.000847
2,0.124330,0.874710,0.000960,0.000248,0.999040,0.029584,0.004924,0.000512,3681.928229,0.000847
3,0.180508,0.818550,0.000941,0.000248,0.999059,0.029577,0.004925,0.000517,4059.958026,0.000847
4,0.220828,0.778263,0.000909,0.000249,0.999091,0.029796,0.004971,0.000511,4672.880330,0.000902


## Now save that in scenario B folder

In [9]:
df.to_csv(FILE_PATH_ScenarioB+'/ScenarioB_100NNFeatures.txt', index= False)

In [10]:
# Clean variables I domnt need right now

df = None
corr_matrix = None
X = None
y = None
data = None 